## Filter MACS images with individual parameters and export to tif

### by Jennika Hammar


This script runs an individual offset for DSNU for each MACS image, performs the devignetting and export the images as tif to a folder.

Preparations:
1. calculate individual offsets for each image (ask Inge for script) and put the csv file where you can find it
2. make a copy of mipps.exe which is found in C:\Program Files\DLR MACS-Box\tools\MACS and 
put it in C:\Program Files\DLR MACS-Box\bin
note: the mipps.exe will disappear from the bin folder after a while and need to be put there again..
----->>>>> need to find a better solution for this!!!!!!!!!!!

3. make a copy of the MIPPS script (for the NIR camera it is 33552.mipps) and put it somewhere where you find it

4. open the copied MIPPS script in notepad and replace name="offset" value="0"/> to name="offset" value="myoffset"/>
and save




In [1]:
# import libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import re
import subprocess
import glob

In [2]:
# create list of all MACS images 
macs_list = glob.glob('D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50/*.macs', recursive=True)

# sort the list
macs_list.sort()

# control that the list is in ascending order
print(macs_list[:10])

['D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00967_012548208_750.macs', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00968_012548608_750.macs', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00969_012549008_750.macs', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00970_012549408_750.macs', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00971_012549808_750.macs', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00972_012550208_750.macs', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00973_012550608_750.macs', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00974_012551008_750.macs', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\\00975_012551408_750.macs', 

In [3]:
# set working diectory and import the csv file with the individual offsets
os.chdir('D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata')
fid = 'NIR_image_offset.csv'
df=pd.read_csv(fid, usecols=['offset', 'NirImage'], sep=',') #usecols=['NirImage', 'offset']
df.head(5) # control that the order correspond to the macs_list order

,NirImage,offset
0,00967_012548208_750.tif,-5000.0
1,00968_012548608_750.tif,-5000.0
2,00969_012549008_750.tif,-5000.0
3,00970_012549408_750.tif,-5000.0
4,00971_012549808_750.tif,-5000.0


In [4]:
# convert to integer (neeced for the MIPPS script)
df=pd.read_csv(fid, usecols=['offset'], sep=',')
df = df.astype(int)

In [5]:
# create list of the offsets
df_list = df.values.tolist()

In [6]:
# remove square brackets around values
df_list = [val for sublist in df_list for val in sublist]
print(df_list[:10])

[-5000, -5000, -5000, -5000, -5000, -5000, -5000, -5000, -5000, -5000]


In [7]:
# change working directory to where the mipps.exe is
os.chdir("C:/Program Files/DLR MACS-Box/bin")

# open the MIPPS script
filename = "D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/33552_all_taps_2018-09-26_12-58-15_modelbased.mipps"
with open(filename, 'r') as c:
    text = c.read()

# define an output file
output = "D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/output.mipps"


# replace 'myoffset' with the offset values from the list and process with the MIPPS script
for i, a in zip(df_list, macs_list):
    print(i,a)
    if 'myoffset' in text:
        b = text.replace('myoffset', str(i))
        with open(output, 'w') as f:
            f.write(b)
            f.close()
            subprocess.run(['mipps.exe', '-config', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/output.mipps',
                            '-output-dir', 'D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/tif/20190606_MACS-Polar18_flight-line-corr/NIR_33552',
                            a], shell=True, capture_output=True)
            
        # Feedback to user of replacement value
            print(f'Replaced cat with value {i}')


-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\00967_012548208_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\00968_012548608_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\00969_012549008_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\00970_012549408_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\00971_012549808_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\00972_012550208_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\00973_012550608_750.macs
Replaced cat with value -5000

Replaced cat with value 0
-328 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01026_012611808_750.macs
Replaced cat with value -328
-1314 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01027_012612208_750.macs
Replaced cat with value -1314
-2147 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01028_012612608_750.macs
Replaced cat with value -2147
-2877 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01029_012613008_750.macs
Replaced cat with value -2877
-3361 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01030_012613408_750.macs
Replaced cat with value -3361
-3548 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01031_012613808_750.macs
Replaced cat with value -3548
-3653 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01032_012614208_750.macs
Repla

Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01085_012635408_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01086_012635808_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01087_012636208_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01088_012636608_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01089_012637008_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01090_012637408_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\01091_012637808_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processi

Replaced cat with value -3621
-2328 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02466_013547801_750.macs
Replaced cat with value -2328
-1050 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02467_013548201_750.macs
Replaced cat with value -1050
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02468_013548601_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02469_013549001_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02470_013549401_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02471_013549801_750.macs
Replaced cat with value 0
0 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02472_013550201_750.macs
Replaced cat with value 0
0 D:/00_o

Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02524_013611001_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02525_013611401_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02526_013611801_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02527_013612201_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02528_013612601_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02529_013613001_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\02530_013613401_750.macs

...

Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13643_025018547_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13644_025018947_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13645_025019347_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13646_025019747_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13647_025020147_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13648_025020547_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13649_025020947_750.macs

Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13701_025041746_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13702_025042146_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13703_025042546_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13704_025042946_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13705_025043346_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13706_025043746_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\13707_025044146_750.macs

Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15247_030100139_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15248_030100539_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15249_030100939_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15250_030101339_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15251_030101739_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15252_030102139_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15253_030102539_750.macs

Replaced cat with value -4667
-4619 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15305_030123339_750.macs
Replaced cat with value -4619
-4588 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15306_030123739_750.macs
Replaced cat with value -4588
-4636 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15307_030124138_750.macs
Replaced cat with value -4636
-4952 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15308_030124538_750.macs
Replaced cat with value -4952
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15309_030124939_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15310_030125338_750.macs
Replaced cat with value -5000
-5000 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15311_030125738_750.macs

Replaced cat with value -3684
-3262 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15363_030146538_750.macs
Replaced cat with value -3262
-3123 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15364_030146938_750.macs
Replaced cat with value -3123
-2975 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15365_030147338_750.macs
Replaced cat with value -2975
-2547 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15366_030147738_750.macs
Replaced cat with value -2547
-2123 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15367_030148138_750.macs
Replaced cat with value -2123
-1901 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15368_030148538_750.macs
Replaced cat with value -1901
-1693 D:/00_orga/12_MACS_processing/01_anaktuvuk_chuck/01_rawdata/macs/33552_Cam-NIR-50\15369_030148938_750.macs

In [8]:
# run this to see the different options for the MIPPS
subprocess.run(['mipps.exe', '-h'], shell=True, capture_output=True)

CompletedProcess(args=['mipps.exe', '-h'], returncode=1, stdout=b'\r\nThis is MIPPS - Matrix camera Image PreProcessing Suite (0.8.6365ed1a build: Wed 21/17/03)\r\n(C) 2012-2014 by DLR / OS-ASK\r\n\r\nUsage: mipps.exe [OPTIONS] image files or directory\r\n OPTIONS:\r\n -help, -h            show this help text \r\n -version             show version information \r\n -verbose, -v         show additional information during processing \r\n -config=x, -c=x      specify a MIPPS config file describing the filters to be \r\n                      applied \r\n -output-dir=x, -o=x  choose target directory for the processed files (by \r\n                      default output is written to the current directory.) \r\n -logfile=x, -l=x     Redirect all log messages to this file. \r\n -terminal, -t        Force log output to the terminal (even if file logging is \r\n                      enabled). \r\n -jobs=x, -j=x        Number of jobs to run in parallel. \r\n -omp=x, -m=x         Low level parallele